# Folders setup

In [4]:
models_folder = './models/aimet_balanced/'
aimet_ori = models_folder + 'BED_classifier__best_mean_F1__AIMET_Balanced__BIPOLAR_Out__QONNX.onnx'

# Model Clean

In [5]:
from finn.util.visualization import showSrc, showInNetron
from qonnx.util.cleanup import cleanup as qonnx_cleanup

In [6]:
qonnx_clean_filename = models_folder + '01_clean.onnx'
qonnx_cleanup(aimet_ori, out_file=qonnx_clean_filename)

In [7]:
showInNetron(aimet_ori)

Serving './models/aimet_balanced/BED_classifier__best_mean_F1__AIMET_Balanced__BIPOLAR_Out__QONNX.onnx' at http://0.0.0.0:8083


In [8]:
showInNetron(qonnx_clean_filename)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/01_clean.onnx' at http://0.0.0.0:8083


# Compare All Outputs

In [9]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe

import numpy as np

In [13]:
test_ip = np.random.randint(low=0, high=256, size=(1, 3, 230, 230)) / 255.
test_ip = test_ip.astype(np.float32)

In [14]:
clean_model = ModelWrapper(qonnx_clean_filename)

In [15]:
input_dict = {"global_in": test_ip}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[-1., -1.]], dtype=float32)

# Convert to FINN

In [16]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

In [17]:
model = ModelWrapper(qonnx_clean_filename)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())

In [18]:
finn_tidy = models_folder + '02_finn_tidy.onnx'
model.save(finn_tidy)

In [19]:
showInNetron(finn_tidy)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/02_finn_tidy.onnx' at http://0.0.0.0:8083


# PreProcess

In [22]:
import torch
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from brevitas.export import export_qonnx

In [23]:
model = ModelWrapper(finn_tidy)
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = models_folder + "./prepro_node.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/merge_onnx_models.py:70: UserWarning: [MergeONNXModels] opsets for models to merge differ: 14 vs 17, output model will use opset 17
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [24]:
from qonnx.transformation.infer_datatypes import InferDataTypes

### Save prepro again after tidy

In [25]:
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

In [26]:
finn_prepro = models_folder + '03_finn_prepro.onnx'
model.save(finn_prepro)

In [27]:
showInNetron(finn_prepro)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/03_finn_prepro.onnx' at http://0.0.0.0:8083


# Streamline

In [28]:
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.change_datalayout import ChangeDataLayoutQuantAvgPool2d
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

from finn.transformation.streamline import Streamline
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants

In [29]:
model = ModelWrapper(finn_prepro)
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(ChangeDataLayoutQuantAvgPool2d())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

/home/gmoreno/uav/finn/src/finn/transformation/streamline/absorb.py:166: RuntimeWarning: divide by zero encountered in divide
  Tnew = T / A.reshape(-1, 1)


In [30]:
finn_streamline = models_folder + '04_finn_streamline.onnx'
model.save(finn_streamline)

In [31]:
showInNetron(finn_streamline)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/04_finn_streamline.onnx' at http://0.0.0.0:8083


# HW Layers

In [32]:
from finn.util.basic import pynq_part_map
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [33]:
print(pynq_part_map)
print(fpga_part)

{'Ultra96': 'xczu3eg-sbva484-1-e', 'Ultra96-V2': 'xczu3eg-sbva484-1-i', 'Pynq-Z1': 'xc7z020clg400-1', 'Pynq-Z2': 'xc7z020clg400-1', 'ZCU102': 'xczu9eg-ffvb1156-2-e', 'ZCU104': 'xczu7ev-ffvc1156-2-e', 'ZCU111': 'xczu28dr-ffvg1517-2-e', 'RFSoC2x2': 'xczu28dr-ffvg1517-2-e', 'RFSoC4x2': 'xczu48dr-ffvg1517-2-e', 'KV260_SOM': 'xck26-sfvc784-2LV-c'}
xc7z020clg400-1


In [34]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten

from qonnx.custom_op.registry import getCustomOp

### Set Datatype of Multithresholds that were not infered properly

All MultiThresholds following a convolution splitted by Spatial SVD are not inferred properly.
- Conv1 (3,1) -> QuantIdentity -----------------> NOT OK
- Conv2 (1,3) -> BatchNorm -> ReLU -------------> OK

In [35]:
model = ModelWrapper(finn_streamline)

In [36]:
Multithreshold_node = model.get_nodes_by_op_type("MultiThreshold") 

In [37]:
for node in Multithreshold_node:
    if model.get_tensor_datatype(node.input[1]) == "FLOAT32":
        print(f'Node with Float32 annotation {node.name}')
        model.set_tensor_datatype(node.input[1], DataType["INT32"])
        print(model.get_tensor_datatype(node.input[1]))

Node with Float32 annotation MultiThreshold_2
INT32
Node with Float32 annotation MultiThreshold_5
INT32
Node with Float32 annotation MultiThreshold_8
INT32
Node with Float32 annotation MultiThreshold_16
INT32


In [38]:
finn_set_datatype = models_folder + '05_finn_set_datatype.onnx'

In [39]:
model.save(finn_set_datatype)

In [40]:
showInNetron(finn_set_datatype)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/05_finn_set_datatype.onnx' at http://0.0.0.0:8083


### Change last Bipolar Node to Binary

In [87]:
model = ModelWrapper(finn_set_datatype)

In [88]:
Multithreshold_node = model.get_nodes_by_op_type("MultiThreshold") 

In [66]:
# for node in Multithreshold_node:
#     if node.name == "MultiThreshold_19":
#         node_inst = getCustomOp(node)
#         print(node_inst.get_nodeattr("out_scale"))
#         node_inst.set_nodeattr("out_dtype", "BINARY")
#         node_inst.set_nodeattr("out_scale", 1.0)
#         node_inst.set_nodeattr("out_bias", 0.0)
#         print(node)

2.0
input: "MatMul_18_out0"
input: "MultiThreshold_19_param0"
output: "global_out"
name: "MultiThreshold_19"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "BINARY"
  type: STRING
}
attribute {
  name: "out_scale"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "out_bias"
  f: 0.0
  type: FLOAT
}
domain: "qonnx.custom_op.general"



In [90]:
for node in Multithreshold_node:
    node_inst = getCustomOp(node)
    if node_inst.get_nodeattr("out_dtype") == "BIPOLAR":
        node_inst.set_nodeattr("out_dtype", "BINARY")
        node_inst.set_nodeattr("out_scale", 1.0)
        node_inst.set_nodeattr("out_bias", 0.0)
        print(node)

input: "MatMul_18_out0"
input: "MultiThreshold_19_param0"
output: "global_out"
name: "MultiThreshold_19"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "BINARY"
  type: STRING
}
attribute {
  name: "out_scale"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "out_bias"
  f: 0.0
  type: FLOAT
}
domain: "qonnx.custom_op.general"



In [91]:
global_out_name = model.graph.output[0].name
global_out_name

'global_out'

In [92]:
model.set_tensor_datatype(global_out_name, DataType["BINARY"])

In [93]:
finn_bipolar_to_binary = models_folder + '06_finn_bipolar_to_binary.onnx'

In [94]:
model.save(finn_bipolar_to_binary)

In [95]:
showInNetron(finn_bipolar_to_binary)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/06_finn_bipolar_to_binary.onnx' at http://0.0.0.0:8083


### Standlone Thresholds

In [96]:
model = ModelWrapper(finn_bipolar_to_binary)

model = model.transform(to_hw.InferThresholdingLayer())

In [97]:
finn_std_alone_thres = models_folder + '07_finn_std_alone_thres.onnx'

In [98]:
model.save(finn_std_alone_thres)

In [99]:
showInNetron(finn_std_alone_thres)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/07_finn_std_alone_thres.onnx' at http://0.0.0.0:8083


### Rest of the Streamline Process

In [100]:
model = ModelWrapper(finn_std_alone_thres)
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())

# input quantization (if any) to standalone thresholding
model = model.transform(to_hw.InferThresholdingLayer())
model = model.transform(to_hw.InferPool())
model = model.transform(to_hw.InferStreamingMaxPool())
model = model.transform(to_hw.InferConvInpGen())

# get rid of Reshape(-1, 1) operation between hw nodes 
model = model.transform(RemoveCNVtoFCFlatten())

# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())

# infer tensor data layouts
model = model.transform(InferDataLayouts())

model = model.transform(Streamline())

In [101]:
finn_hw_layers = models_folder + '08_fin_hw_layers.onnx'
model.save(finn_hw_layers)

In [102]:
showInNetron(finn_hw_layers)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/08_fin_hw_layers.onnx' at http://0.0.0.0:8083


# Dataflow Partition

In [103]:
model = ModelWrapper(finn_hw_layers)
parent_model = model.transform(CreateDataflowPartition())

In [104]:
finn_parent_filename = models_folder + '00_finn_dataflow_parent.onnx'
parent_model.save(finn_parent_filename)

In [105]:
showInNetron(finn_parent_filename)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/00_finn_dataflow_parent.onnx' at http://0.0.0.0:8083


In [106]:
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_filename = sdp_node.get_nodeattr("model")
dataflow_model = ModelWrapper(dataflow_filename)

# Specialize Layers

In [107]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers

### There are no Padding Nodes in Balanced Model (Aimet No Padding Balanced)

In [108]:
FMPadding_node = dataflow_model.get_nodes_by_op_type("FMPadding")
i = 0
for node in FMPadding_node:
    node_inst = getCustomOp(node)
    node_inst.set_nodeattr("preferred_impl_style", "hls")
    print(f'Node {i}: {node}')
    i += 1

In [109]:
# save the dataflow partition with a different name for easier access
# and specialize the layers to HLS variants
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))

dataflow_model = dataflow_model.transform(GiveUniqueNodeNames())
dataflow_model = dataflow_model.transform(GiveReadableTensorNames())

finn_dataflow_filename = models_folder + '20_finn_dataflow_model.onnx'
dataflow_model.save(finn_dataflow_filename)

In [110]:
showInNetron(finn_dataflow_filename)

Stopping http://0.0.0.0:8083
Serving './models/aimet_balanced/20_finn_dataflow_model.onnx' at http://0.0.0.0:8083


### Check execution???

In [50]:
# parent_dataflow_model = ModelWrapper(finn_parent_filename)

In [51]:
# input_dict = {"global_in": test_ip*255}
# output_dict = oxe.execute_onnx(parent_dataflow_model, input_dict)
# produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
# produced_clean_qonnx

# Folding Factors

In [52]:
from finn.transformation.fpgadataflow.set_folding import SetFolding

**Taregt Cycles Per Frame**

If target is 25 FPS, inference time is $\frac{1}{25}=40ms$

If $clk = 10 ns$:
$$
Target~Cycles~Per~Frame = \frac{40\times 10^{-3}}{10\times 10^{-9}}= 4\times 10^{6}
$$

No se tiene en cuenta el tiempo de preprocesado, que en realidad debería ser inexistente, ya que está embebido en el preprocess del modelo. 

In [53]:
# folder = SetFolding(
#     target_cycles_per_frame=4000000,#1000,
#     mvau_wwidth_max=36,
#     two_pass_relaxation=True
# )

In [54]:
model = ModelWrapper(finn_dataflow_filename)

apply method of SetFolding returns (model, False), so model is [0]

maybe it is easier to do: model, _ = folder.apply(...)

In [55]:
# model = folder.apply(model)

model = model.transform(SetFolding(
    target_cycles_per_frame=4000000,#1000,
    mvau_wwidth_max=36,
    two_pass_relaxation=True)
)

In [56]:
folding_filename = models_folder + '30_finn_folding.onnx'
#model[0].save(folding_filename)
model.save(folding_filename)

In [57]:
showInNetron(folding_filename)

Stopping http://0.0.0.0:8083
Serving 'experiments/01_w2a2/models/30_finn_folding.onnx' at http://0.0.0.0:8083


### Check Total Estimated Cycles, looping over each node attribute

In [58]:
all_nodes = model.get_finn_nodes()

In [59]:
i = 0
total_cycles = []
for node in all_nodes:
    my_node = getCustomOp(node)
    node_cycles = my_node.get_nodeattr("cycles_estimate")
    total_cycles.append(node_cycles)
    print(f'Node {i} estimated cycles: {node_cycles}')
    i += 1
print(f'\nTotal estimated cycles: {np.array(total_cycles).sum()}')

Node 0 estimated cycles: 150528
Node 1 estimated cycles: 153228
Node 2 estimated cycles: 1356117
Node 3 estimated cycles: 1605632
Node 4 estimated cycles: 2008896
Node 5 estimated cycles: 1605632
Node 6 estimated cycles: 415872
Node 7 estimated cycles: 3620064
Node 8 estimated cycles: 3612672
Node 9 estimated cycles: 251360
Node 10 estimated cycles: 200704
Node 11 estimated cycles: 802816
Node 12 estimated cycles: 53824
Node 13 estimated cycles: 453488
Node 14 estimated cycles: 3612672
Node 15 estimated cycles: 3211264
Node 16 estimated cycles: 107648
Node 17 estimated cycles: 906976
Node 18 estimated cycles: 3612672
Node 19 estimated cycles: 257392
Node 20 estimated cycles: 200704
Node 21 estimated cycles: 1605632
Node 22 estimated cycles: 28800
Node 23 estimated cycles: 227808
Node 24 estimated cycles: 3612672
Node 25 estimated cycles: 1605632
Node 26 estimated cycles: 28800
Node 27 estimated cycles: 227808
Node 28 estimated cycles: 3612672
Node 29 estimated cycles: 1605632
Node 30 e

# Minimize 

In [60]:
from finn.transformation.fpgadataflow.minimize_accumulator_width import (
    MinimizeAccumulatorWidth,
)
from finn.transformation.fpgadataflow.minimize_weight_bit_width import (
    MinimizeWeightBitWidth,
)

In [61]:
model = ModelWrapper(folding_filename)

In [62]:
model = model.transform(MinimizeAccumulatorWidth())
model = model.transform(MinimizeWeightBitWidth())

In [63]:
minimize_filename = models_folder + '31_finn_minimize.onnx'
model.save(minimize_filename)

In [64]:
showInNetron(minimize_filename)

Stopping http://0.0.0.0:8083
Serving 'experiments/01_w2a2/models/31_finn_minimize.onnx' at http://0.0.0.0:8083


# HW IP Generation: PrepareIP and HLSSynthIP 

In [65]:
# from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
# from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP

In [66]:
# model = ModelWrapper(minimize_filename)

In [67]:
# model = model.transform(PrepareIP(fpga_part, target_clk_ns))
# model = model.transform(HLSSynthIP())

In [68]:
# hw_filename = models_folder + '32_finn_hw_ipgen.onnx'
# model.save(hw_filename)

In [69]:
# showInNetron(hw_filename)

# FIFO depths

In [70]:
from finn.transformation.fpgadataflow.set_fifo_depths import InsertAndSetFIFODepths

In [71]:
# Default settings
# fifo_depths = InsertAndSetFIFODepths(
#     fpgapart=fpga_part,
#     clk_ns=10.0,
#     max_qsrl_depth=256,
#     max_depth=None,
#     swg_exception=False,#True, # Used to optimize convolution FIFOs, splitting in several with Power of Two
#     vivado_ram_style="auto",
#     force_python_sim=False,
# )

In [72]:
#model = ModelWrapper(hw_filename)

model = ModelWrapper(minimize_filename)

In [73]:
#model = fifo_depths.apply(model)

model = model.transform(InsertAndSetFIFODepths(
    fpgapart=fpga_part,
    clk_ns=10.0,
    max_qsrl_depth=256,
    max_depth=None,
    swg_exception=False,#True, # Used to optimize convolution FIFOs, splitting in several with Power of Two
    vivado_ram_style="auto",
    force_python_sim=False)
)

%Warning-MODDUP: /home/gmoreno/workspace/vivado_stitch_proj_7yak8qkc/finn_design_wrapper.v:23168:8: Duplicate declaration of module: 'dwc_axi'
23168 | module dwc_axi #(
      |        ^~~~~~~
                 /home/gmoreno/workspace/vivado_stitch_proj_7yak8qkc/finn_design_wrapper.v:23100:8: ... Location of original declaration
23100 | module dwc_axi #(
      |        ^~~~~~~
                 ... For warning description see https://verilator.org/warn/MODDUP?v=4.224
                 ... Use "/* verilator lint_off MODDUP */" and lint_on around source to disable this message.
%Warning-MODDUP: /home/gmoreno/workspace/vivado_stitch_proj_7yak8qkc/finn_design_wrapper.v:25289:8: Duplicate declaration of module: 'swg_controller'
25289 | module swg_controller
      |        ^~~~~~~~~~~~~~
                 /home/gmoreno/workspace/vivado_stitch_proj_7yak8qkc/finn_design_wrapper.v:1979:8: ... Location of original declaration
 1979 | module swg_controller
      |        ^~~~~~~~~~~~~~
%Warning-MODDUP

make: Entering directory '/home/gmoreno/workspace/verilator_fifosim_oqcf_yeb'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     --std=c++11  -DVL_THREADED -std=gnu++17 -Os -c -o verilator_fifosim.o verilator_fifosim.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     --std=c++11  -DVL_THREADED -std=gnu++17 -Os -c -o verilated.o /usr/local/share/verilator/include/verilated.cpp
ccache 

In [74]:
fifo_filename = models_folder + '31_finn_fifo.onnx'
#model[0].save(fifo_filename)
model.save(fifo_filename)

In [76]:
showInNetron(fifo_filename)

Stopping http://0.0.0.0:8083
Serving 'experiments/01_w2a2/models/31_finn_fifo.onnx' at http://0.0.0.0:8083


### Streamline FIFOs

In [77]:
from finn.transformation.fpgadataflow.set_fifo_depths import SplitLargeFIFOs
from finn.transformation.fpgadataflow.set_fifo_depths import RemoveShallowFIFOs

In [78]:
#model = model[0].transform(SplitLargeFIFOs())

model = model.transform(SplitLargeFIFOs())
model = model.transform(RemoveShallowFIFOs())

/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/rtl/streamingfifo_rtl.py:70: UserWarning: StreamingFIFO_rtl_1: rounding-up FIFO depth from 116540 to 131072 for impl_style=vivado
  warnings.warn(
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/rtl/streamingfifo_rtl.py:70: UserWarning: StreamingFIFO_rtl_2: rounding-up FIFO depth from 153228 to 262144 for impl_style=vivado
  warnings.warn(
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/rtl/streamingfifo_rtl.py:70: UserWarning: StreamingFIFO_rtl_4: rounding-up FIFO depth from 15637 to 16384 for impl_style=vivado
  warnings.warn(
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/rtl/streamingfifo_rtl.py:70: UserWarning: StreamingFIFO_rtl_5: rounding-up FIFO depth from 34469 to 65536 for impl_style=vivado
  warnings.warn(
/home/gmoreno/uav/finn/src/finn/custom_op/fpgadataflow/rtl/streamingfifo_rtl.py:70: UserWarning: StreamingFIFO_rtl_7: rounding-up FIFO depth from 38684 to 65536 for impl_style=vivado
  warnings.wa

In [ ]:
# after FIFOs are ready to go, call PrepareIP and HLSSynthIP again
# this will only run for the new nodes (e.g. FIFOs and DWCs) -> DWCs for Mobilenet
# model = model.transform(PrepareIP(fpga_part, target_clk_ns))
# model = model.transform(HLSSynthIP())

In [79]:
fifo_streamline_filename = models_folder + '33_finn_fifo_streamline.onnx'
model.save(fifo_streamline_filename)

In [80]:
showInNetron(fifo_streamline_filename)

Stopping http://0.0.0.0:8083
Serving 'experiments/01_w2a2/models/33_finn_fifo_streamline.onnx' at http://0.0.0.0:8083


# PYNQ Driver

In [81]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

In [82]:
model = ModelWrapper(fifo_streamline_filename)
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

/home/gmoreno/uav/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 158 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/home/gmoreno/uav/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:56: UserWarning: Using pre-existing code for StreamingDataflowPartition_1_Thresholding_rtl_0
  warnings.warn("Using pre-existing code for %s" % node.name)
/home/gmoreno/uav/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:56: UserWarning: Using pre-existing code for StreamingDataflowPartition_1_FMPadding_hls_0
  warnings.warn("Using pre-existing code for %s" % node.name)
/home/gmoreno/uav/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:56: UserWarning: Using pre-existing code for StreamingDataflowPartition_1_ConvolutionInputGenerator_rtl_0
  warnings.warn("Using pre-existing code for %s" % node.name)
/home/gmoreno/uav/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:56: UserWarning: Using pre-existing code for Streami

Exception: Synthesis failed, no bitfile found. Check logs under /home/gmoreno/workspace/vivado_zynq_proj_bumup3lp

In [ ]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver

In [ ]:
model = model.transform(MakePYNQDriver("zynq-iodma"))

In [ ]:
pynq_driver_filename = '40_pynq_driver.onnx'
model.save(pynq_driver_filename)